# Классы

In [1]:
import numpy as np
from numba import njit

In [2]:
class ZeroCrossingSignal:

    def __init__(self, signal, harmonic_hf, start_offset, start_period):
        self.__amplitudes, self.__phases, self.__biases = self.zero_crossing(signal, harmonic_hf, start_offset, start_period)

    @staticmethod
    @njit
    def zero_crossing(signal, harmonic_hf, start_offset, start_period):
        offset = start_offset
        period = start_period

        amplitudes = [1.,]
        phases = [1.,]
        biases = [1.,]

        amplitudes.pop()
        phases.pop()
        biases.pop()

        harmonics_vector = np.array([0., 0., 0.])
        iteration = 0

        while offset < (len(signal) - np.floor(period)-1):
            if iteration == 100: break
            phase = 2 * np.pi / period * harmonic_hf # связано со временеи обората, длинна сгустка - bunch ## похоже на фазу 
 
            harmonics_vector[0] = np.sum(signal[offset:offset + int(period) + 1] * np.cos(phase * np.arange(int(period) + 1)))
            harmonics_vector[1] = np.sum(signal[offset:offset + int(period) + 1] * np.sin(phase * np.arange(int(period) + 1)))
            harmonics_vector[2] = np.sum(signal[offset:offset + int(period) + 1])
 
            P_b = np.array(((0., 0., 0.),
                            (0., 0., 0.),
                            (0., 0., 0.)))

            cos_vals = np.cos(phase * np.arange(int(period) + 1))
            sin_vals = np.sin(phase * np.arange(int(period) + 1))

            P_b[0, 0] = np.sum(cos_vals**2)
            P_b[0, 1] = np.sum(cos_vals * sin_vals)
            P_b[0, 2] = np.sum(cos_vals)
            P_b[1, 0] = P_b[0, 1]
            P_b[1, 1] = np.sum(sin_vals**2)
            P_b[1, 2] = np.sum(sin_vals)
            P_b[2, 0] = P_b[0, 2]
            P_b[2, 1] = P_b[1, 2]
            P_b[2, 2] = int(period) + 1

            A = np.linalg.inv(P_b) @ harmonics_vector.T # ----
     
            amplitudes.append(np.sqrt(A[0]**2 + A[1]**2)) # амплитуда 
            phases.append(offset + (1 / phase * np.arctan2(A[0], -A[1]))) # фаза 
            biases.append(A[2]) # ----

            offset = int(phases[iteration] + period) # перечет офсета 

            period = 1 / 10 * (phases[iteration] - phases[iteration-10]) if iteration > 12 else period # перечсет периода для окна 

            iteration += 1

        return np.array(amplitudes), np.array(phases), np.array(biases)
        
    @property
    def amplitudes(self): return self.__amplitudes

    @property
    def phases(self): return self.__phases

    @property
    def biases(self): return self.__biases

In [3]:
class LorentzFactors:
    def __init__(self, phases, booster_perimeter, delta_time, speed_of_light=2.997925e10, proton_rest_mass = 938.256e6):
        self.__phase_difference = np.diff(phases)
        self.__beta = booster_perimeter / (speed_of_light * delta_time * self.phase_difference)
        self.__gamma = 1 / np.sqrt(1 - self.beta**2)
        self.__energy = (self.gamma-1) * proton_rest_mass

    @property
    def phase_difference(self): return self.__phase_difference

    @property
    def beta(self): return self.__beta

    @property
    def gamma(self): return self.__gamma

    @property
    def energy(self): return self.__energy

In [4]:
class ParametersFCTRF:
    def __init__(self, signal_fct, signal_rf, period, offset_fct, offset_rf, offset_between_signals,
                 harmonic_hf, booster_perimeter, delta_time, averaging_window_for_finding_fct_minimums,
                 averaging_window_for_phases, phase_offset=0, charge_number = 28, proton_rest_mass = 938.256e6,
                 electron_charge = 1.6021e-19, speed_of_light = 2.997925e10):
        self.__zero_crossing_rf = ZeroCrossingSignal(signal_rf, harmonic_hf, offset_rf, period)
        self.__phases = self.__zero_crossing_rf.phases
        self.__lorentz_factors_rf = LorentzFactors(self.zero_crossing_rf.phases, booster_perimeter, delta_time, speed_of_light, proton_rest_mass)
        self.__discretized_phases = np.floor(self.zero_crossing_rf.phases[1:]).astype(int)
        self.__discretized_phases_with_offset = self.discretized_phases[phase_offset:]
        self.__fct_minimums = self.signal_minimums_with_averaging(signal_fct[offset_between_signals:], self.discretized_phases_with_offset,
                                                                averaging_window_for_finding_fct_minimums)
        self.__corrected_phases = self.correct_phases(signal_fct[offset_between_signals:], self.fct_minimums, self.zero_crossing_rf.phases,
                                                      phase_offset, harmonic_hf)
        self.__averaged_phases = np.convolve(np.hstack((np.zeros(phase_offset), self.corrected_phases)), #Зачем-то добавляется два нуля...
                                    np.ones(averaging_window_for_phases)/averaging_window_for_phases,
                                    mode='valid')[::averaging_window_for_phases][:-1] #Зачем-то откидывается последнее...

        self.__intensities = self.find_intensities(signal_fct[offset_between_signals:], self.fct_minimums, self.discretized_phases_with_offset,
                                        delta_time, charge_number=charge_number, electron_charge=electron_charge)       
    
    @property
    def phases(self): return self.__phases

    @property
    def zero_crossing_rf(self): return self.__zero_crossing_rf

    @property
    def lorentz_factors_rf(self): return self.__lorentz_factors_rf

    @property
    def discretized_phases(self): return self.__discretized_phases

    @property
    def discretized_phases_with_offset(self): return self.__discretized_phases_with_offset

    @property
    def fct_minimums(self): return self.__fct_minimums

    @property
    def corrected_phases(self): return self.__corrected_phases

    @property
    def averaged_phases(self): return self.__averaged_phases

    @property
    def intensities(self): return self.__intensities

    @staticmethod
    def correct_phases(signal, signal_minimums, phases, phase_offset, harmonic_hf):
        discretized_phases = np.int32(phases[1:])
        corrected_phases = np.zeros(len(discretized_phases) - phase_offset - 1, dtype=float)

        for i in range(len(corrected_phases)):
            up_sum = np.sum((signal[discretized_phases[phase_offset+i]: discretized_phases[phase_offset+i+1]+1] - signal_minimums[i])*\
                            (np.arange(discretized_phases[phase_offset+i+1]-discretized_phases[phase_offset+i]+1) + discretized_phases[phase_offset+i] -\
                               (phases[phase_offset+i+2] + phases[phase_offset+i+1]) / 2))
            under_sum = np.sum(signal[discretized_phases[phase_offset+i]:
                                    discretized_phases[phase_offset+i+1]+1] - signal_minimums[i])
            corrected_phases[i] = 360 * harmonic_hf / (discretized_phases[phase_offset+i+1] - discretized_phases[phase_offset+i]) * (up_sum / under_sum)
        
        return corrected_phases

    @staticmethod
    def find_intensities(signal, signal_minimums, phases, delta_time, charge_number = 28, electron_charge=1.6021e-19):
        return np.fromiter((8.6 * 10**(-5) * (10**(-3) / (5 * charge_number)) *\
            (delta_time / electron_charge) * np.sum(signal[phases[i]: phases[i+1] + 1] - signal_minimums[i])
            for i in range(len(phases)-1)), float) / 10 ## НЕ ДО КОНЦА ПОНЯЛ, ПОЧЕМУ НАДО ДЕЛИТЬ НА ДЕСЯТЬ, МБ ОШИБКА В ДРУГОМ МЕСТЕ 

    @staticmethod
    def signal_minimums_with_averaging(signal, period_nodes, window_length):
        result = np.zeros(len(period_nodes)-1)
        for i in range(len(period_nodes)-1):
            result[i] = np.convolve(signal[period_nodes[i]: period_nodes[i+1]], np.ones(window_length) / window_length, mode='valid').min()
        return result

# Константы

In [5]:
mass_proton = 938.256 * 10**6 # масса покоя протона 
charge_electron  = 1.6021 * 10**(-19) # заряд электрона в Кл
speed_of_light = 2.997925 * 10**10

# Параметры

In [6]:
booster_perimeter = 21096 # периметр бустера в мм #ВЕЗДЕ ОДИН
delta_time = (50 * 10e5)**(-1) #ВЕЗДЕ ОДНО

# оффсеты по какой-то причине отличаются у некоторых, это нужно уточнить
offset_fct = 45 #нафиг не нужен в классе

#ОБЩИЕ ПАРАМЕТРЫ
booster_perimeter = 21096 # периметр бустера в мм
delta_time = (5e6)**(-1) 
offset_between_signals = 10
data_length = 7 * 32 * 128 * 1024 + 1
phase_offset = 2
averaging_window_for_finding_fct_minimums = 13
averaging_window_for_phases = 20
bytes_to_skip_for_second_half = 68_000_000

In [7]:
half_names = 'first_half', 'second_half'
period = {'first_half': 425.067, 'second_half': 127.848}
harmonic_hf = {'first_half': 5, 'second_half': 1}
offset_rf = {'first_half': 10, 'second_half': 112}

In [8]:
signals_fct_filepaths = ['data/booster_acceleration/27_01_23_booster_fct_1.bin', 'data/booster_acceleration/27_01_23_booster_fct_2.bin']
signals_rf_filepaths = ['data/booster_acceleration/27_01_23_booster_rf_1.bin', 'data/booster_acceleration/27_01_23_booster_rf_2.bin']

In [9]:
signals_rf = []
signals_fct = []
for signal_fct_filepath, signal_rf_filepath in zip(signals_fct_filepaths, signals_rf_filepaths):
    signals_rf.append({'first_half': np.fromfile(signal_rf_filepath, dtype='int16')[:data_length],
                      'second_half': np.fromfile(signal_rf_filepath, dtype='int16', offset=bytes_to_skip_for_second_half)[:data_length]})

    signals_fct.append({'first_half': np.fromfile(signal_fct_filepath, dtype='int16')[:data_length],
                       'second_half': np.fromfile(signal_fct_filepath, dtype='int16', offset=bytes_to_skip_for_second_half)[:data_length]})

In [10]:
parameters_fct_rf = []
for i in range(len(signals_rf)):
    dict_to_append = {}
    for key in half_names:
        dict_to_append[key] = ParametersFCTRF(signal_fct=signals_fct[i][key], signal_rf=signals_rf[i][key], period=period[key],
                                             offset_fct=offset_fct, offset_rf=offset_rf[key], offset_between_signals=offset_between_signals,
                                             harmonic_hf=harmonic_hf[key], booster_perimeter=booster_perimeter, delta_time=delta_time,
                                             phase_offset=phase_offset, averaging_window_for_finding_fct_minimums=averaging_window_for_finding_fct_minimums,
                                             averaging_window_for_phases=averaging_window_for_phases)
    parameters_fct_rf.append(dict_to_append)

In [14]:
parameters_fct_rf[0]['first_half'].phases

array([-3.23500015e+01,  3.92770900e+02,  8.17756297e+02,  1.24285215e+03,
        1.66791973e+03,  2.09298520e+03,  2.51805278e+03,  2.94311915e+03,
        3.36818686e+03,  3.79325414e+03,  4.21832001e+03,  4.64338835e+03,
        5.06845596e+03,  5.49351879e+03,  5.91859022e+03,  6.34365453e+03,
        6.76872205e+03,  7.19378984e+03,  7.61885627e+03,  8.04392313e+03,
        8.46899243e+03,  8.89405888e+03,  9.31912434e+03,  9.74419266e+03,
        1.01692585e+04,  1.05943265e+04,  1.10193930e+04,  1.14444589e+04,
        1.18695290e+04,  1.22945932e+04,  1.27196634e+04,  1.31447283e+04,
        1.35697955e+04,  1.39948617e+04,  1.44199284e+04,  1.48449966e+04,
        1.52700635e+04,  1.56951305e+04,  1.61201983e+04,  1.65452655e+04,
        1.69703304e+04,  1.73953971e+04,  1.78204637e+04,  1.82455328e+04,
        1.86705987e+04,  1.90956663e+04,  1.95207339e+04,  1.99458005e+04,
        2.03708670e+04,  2.07959349e+04,  2.12210037e+04,  2.16460694e+04,
        2.20711355e+04,  

In [16]:
import pandas as pd

with open('Acceleration in Booster measured with FCT/BoosterAcceleration1_1half.dat', "rb") as file:
    parameters_from_dan = [
        {'first_half': pd.DataFrame(np.load(file).reshape(-1,3), columns=['Phases', 'Intensities', 'Corrected phases'])}]

with open('Acceleration in Booster measured with FCT/BoosterAcceleration1_2half.dat', "rb") as file:
    parameters_from_dan[0]['second_half'] = pd.DataFrame(np.load(file).reshape(-1,3), columns=['Phases', 'Intensities', 'Corrected phases'])

with open('Acceleration in Booster measured with FCT/BoosterAcceleration2_1half.dat', "rb") as file:
    parameters_from_dan.append({'first_half': pd.DataFrame(np.load(file).reshape(-1,3), columns=['Phases', 'Intensities', 'Corrected phases'])})

with open('Acceleration in Booster measured with FCT/BoosterAcceleration2_2half.dat', "rb") as file:
    parameters_from_dan[1]['second_half'] = pd.DataFrame(np.load(file).reshape(-1,3), columns=['Phases', 'Intensities', 'Corrected phases'])

In [17]:
parameters_from_file = []
for p in parameters_fct_rf:
    dict_to_add = {}
    for key in half_names:
        dict_to_add[key] = pd.DataFrame(
            {'Phases': p[key].phases[phase_offset: -phase_offset],
            'Intensities': p[key].intensities,
            'Corrected phases': p[key].corrected_phases})
    parameters_from_file.append(dict_to_add)

In [19]:
for i in range(len(parameters_from_file)):
    for key in half_names:
        print(i, key)
        print((parameters_from_dan[i][key] - parameters_from_file[i][key]).max(), '\n')

TypeError: 'list' object cannot be interpreted as an integer